In [2]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample
from math import log

In [3]:
# Multilayer nnet spaces
max_layers = 4
hyperspace = []

# sets up the neural network
for num_layers in range(1,max_layers):
    activs = {}
    num_hids = {}

    # set the activation function choice per layer
    for i in range(num_layers):
        activs['activ_%i'%i] = hp.choice('activ',['sigmoid','reLU'])
        num_hids['num_hid_%i'%i] = hp.qloguniform('num_hid',log(10),log(1000),1)
    
    hyperspace.append([
        {'mln_params':[{'activs':activs},
                       {'num_hids':num_hids},
                       {'l1_reg':hp.choice('l1_choice',
                       	[
                    	 	None, 
                         	{'l1_decay':hp.loguniform('lambda',log(1e-5),log(10.))}
                     	])
                       },
                	   
                	   {'l2_reg':hp.choice('l2_choice',
                	   	[
                		 	None, 
                		 	{'l2_decay':hp.loguniform('lambda',log(1e-5),log(10.))}
            		 	])
                	   },

                       {'dropout': hp.choice('dropout',
                       	[
                       		None, 
                    	 	{'input_p':hp.uniform('ip',0,1),'hidden_p':hp.uniform('hp',0,1)}
                	 	])
                       }
                      ]
        },
        {'optim_params':[{'init_method':hp.choice('method',
    						[
    							'gauss',
    							'fan-io'
    						])
        				 },
                        {'scale_factor':hp.loguniform('value',log(1e-3),log(1))},
                        {'rho':hp.uniform('rho',1e-2,0.99)},
                        {'learn_rate':hp.loguniform('learn_rate',log(1e-4),log(10))},
                        {'batch_size':hp.quniform('batch_size',128,1000,1)},
                        {'num_epochs':hp.qloguniform('num_epochs',log(10),log(1e3),1)}]
         }
    ])

full_space = hp.choice('params',[hyperspace[i-1] for i in range(1,max_layers)])
sample(full_space)

({'mln_params': ({'activs': {'activ_0': 'reLU'}},
   {'num_hids': {'num_hid_0': 64.0}},
   {'l1_reg': None},
   {'l2_reg': {'l2_decay': 2.9622507996694027}},
   {'dropout': None})},
 {'optim_params': ({'init_method': 'fan-io'},
   {'scale_factor': 0.09566712111951599},
   {'rho': 0.05984310699669703},
   {'learn_rate': 0.9906889425288772},
   {'batch_size': 935.0},
   {'num_epochs': 256.0})})

In [29]:
sampled_hyperspace = sample(full_space)
sampled_mln_params = {}
sampled_optim_params = {}
for param_type in sampled_hyperspace:
    if 'mln_params' in param_type:
        for d in param_type['mln_params']: sampled_mln_params.update(d)
    elif 'optim_params' in param_type:
        for d in param_type['optim_params']: sampled_optim_params.update(d)

print sampled_mln_params
print sampled_optim_params
                

{'num_hids': {'num_hid_0': 158.0, 'num_hid_1': 317.0}, 'activ_fns': ({'activ_fn_0': 'sigmoid'}, {'activ_fn_1': 'sigmoid'}), 'l1_decay': 0.10323661411876736, 'l2_decay': 0.0}
{'init_method': 'fan-io', 'scale_factor': 0.29352227114962254, 'rho': 0.16125144104294123, 'learn_rate': 0.5195882631000931, 'dropout': {'input_h': 0.2290187233429285, 'input_p': 0.07241995733571027}, 'batch_size': 816.0088150390714}


In [16]:
# test
space = [
hp.choice('activ_layer_1',[['sigmoid'],['reLU']]),
hp.choice('activ_layer_2',[['sigmoid'],['reLU']])
]

In [25]:
sample(space)

(('sigmoid',), ('sigmoid',))

In [18]:
  # Multilayer nnet spaces
max_layers = 4
hyperspace = []
# sets up the neural network
for num_layers in range(1, max_layers):
    activs = [None] * num_layers
    num_hids = [None] * num_layers

    # set the activation function choice per layer
    for i in range(num_layers):
        activs[i] = hp.choice('activ_%i%i' %(num_layers,i), ['sigmoid', 'reLU'])
        num_hids[i] = hp.qloguniform(
            'num_hid_%i%i' %(num_layers,i), log(10), log(3000), 1)

    # define the hyperparamater space to search
    hyperspace.append({'mln_params': [
        {'num_hids': num_hids},
        {'activs': activs},
        {'input_p': hp.uniform('ip', 0, 1)},
        {'hidden_p': hp.uniform('hp', 0, 1)},
        {'l1_reg': hp.choice(
            'l1_lambda', [None, hp.loguniform('l1_decay', log(1e-5), log(10))])},
        {'l2_reg': hp.choice(
            'l2_lambda', [None, hp.loguniform('l2_decay', log(1e-5), log(10))])},
    ],
        'optim_params': [
        {'learn_rate': hp.uniform('learn_rate', 0, 1)},
        {'rho': hp.uniform('rho', 0, 1)},
        {'num_epochs': hp.qloguniform(
            'num_epochs', log(1e2), log(2000), 1)},
        {'batch_size': hp.quniform('batch_size', 128, 1024, 1)},
        {'init_method': hp.choice(
            'init_method', ['gauss', 'fan-io'])},
        {'scale_factor': hp.uniform(
            'scale_factor', 0, 1)}
    ]
    })
full_space = hp.choice('nnet_set',[hyperspace[i-1] for i in range(1,max_layers)])
sample(full_space)

{'mln_params': ({'num_hids': (1273.0, 695.0, 41.0)},
  {'activs': ('reLU', 'sigmoid', 'reLU')},
  {'input_p': 0.8116040489039689},
  {'hidden_p': 0.28727214038203275},
  {'l1_reg': 0.010178150563920199},
  {'l2_reg': None}),
 'optim_params': ({'learn_rate': 0.13473432760266713},
  {'rho': 0.33694127905776916},
  {'num_epochs': 1229.0},
  {'batch_size': 937.0},
  {'init_method': 'gauss'},
  {'scale_factor': 0.17929691316667928})}